## Auto-Encoding Variational Bayes(https://arxiv.org/abs/1312.6114)

Diederik P. Kingma

`Machine Learning Group`
`Universiteit van Amsterdam`
`dpkingma@gmail.com`

Max Welling

`Machine Learning Group`
`Universiteit van Amsterdam`
`welling.max@gmail.com`

---
# 목차
### - Summary
### - Intro
### - Variational inference
### - Reparameterization trick

### - Variational Auto-Encoder : example

---
# 0. Sumarry of VAE

### VAE는 목적 : 
- VAE는 어떠한 데이터(x)에 대해 데이터를 잘 나타내는 continuous한 latent vector(z)가 있을 것이다 라는 가정에서 출발한다.

- Generative model의 목적인 데이터 x의 분포 p(x)를 잘 따라하는 모델을 찾는 것 뿐만 아니라, 좋은 continuous latent vector(z)를 찾는 것을 목표로 한다.

### 특징

- $p(z|x)$를 추론하는 Encoder와 $p(x|z)$를 추론하는 Decoder로 나누어져있다.

- $p(z|x)$를 구하기 힘들기 때문에, Variational Inference의 개념으로, $p(z|x) \simeq q(z)$로 근사한다.

- $p(z|x)$는 Gaussian distribute라 가정하고 mean, variance를 구하고, reparametric trick을 이용하여 latent vector를 sampling 한다.


![01.png](img/01.png)
$$image 출처 : ratsgo.github.io$$

---

# 1. Intro

- How can we perform efficient inference and learning in directed probabilistic models, in the presence of continuous latent variables with intractable posterior distributions and large datasets?

---

### Generative model

- Generative model은 데이터에 대해 X의 분포를 학습하는 모델을 의미한다. 데이터셋의 분포가 $p_{data}(x)$의 분포를 가지고 있다면, Generative model의 분포 $p_{model}(x^{\prime})$ 역시 같은 분포를 가지며, 새로운 데이터 x인 경우를 말한다.


- Generative model은 latenet vecotr($z$) 혹은 잠재변수라고 불리는 변수가 input으로 들어가 데이터 $x^\prime$을 생성한다. 따라서 conditional distribution $p_\theta (x|z)$으로 표현될 수 있다. Generative model은 이러한 분포를 Maximum Likelihood를 통해 학습을 진행한다.


- VAE는 $p_\theta(x|z)$ 뿐만 아니라 $z$의 분포, 특히 $p_\theta(z|x)$를 알고자 하는 것을 목표로 하고 있다.
---

### Problem Senario
![02.png](img/02.png)

---

### goal
![07.png](img/07.png)

---
### Posterior and Bayes rule

- $p(z|x)$ : Posterior(사후확률) : $x$가 나왔을 때, $z$로 부터 나왔을 확률.

- $p(x|z)$ : Likelihood(우도,가능도) : $z$가 주어질 때, $x$가 나올 확률.

- $p(x)$ : Prior(사전확률) : 이미 실험적, 선형적으로 알고 있는 확률.

Posterior(사후확률)은 bayes' rule을 통해 구할 수 있다.

$$p(z|x) = \frac{ p(x|z) p(z) }{p(x)} $$

---
# 2. Variational inference
***(variational inference의 내용은 [ratsgo](https://ratsgo.github.io/generative%20model/2017/12/19/vi/) 블로그를 많이 참고 하였습니다.)***

$p_\theta(z|x)$ 와 $p_\theta(x|z)$ 의 모델을 찾아내는 것의 가장 큰 문제는 $p(x) = \int{p_\theta(z) p_\theta(x|z)dz }$가 intractable 하고, 그로인해 , $p(z|x) = p_\theta(x|z)p_\theta(z) / p_\theta(x) $가 intractable 하다는 것이다. 이러한 점을 해결하는 방법으로 논문에서는 Variational inference를 이용한다.
![03.png](img/03.png)

---
### Variational inference

Variational inference란 다루기 힘든 분포 $p_\theta(z|x)$를 다른 분포 $q_\phi (z|x)$로 근사하는 방법을 뜻한다.
![04.png](img/04.png)
$$ref : ratsgo blog $$


근사한 $q_\phi(z|x)$를 찾기위해, Kullback-Leibler divergence을 사용한다.

$$ arg \min\limits _{\phi} D_{KL}( q(z|x) || p(z|x) ) $$

KLD를 최소화하는 $\phi$를 찾는 문제로 변화하게 되고, 이는 Monte Carlo 방법이나, GSD로 풀 수 있는 문제로 변화된다.


        - *참고 : Kullback-Leibler Divergence (KLD)*
![05](img/05.png)

---
### Variational bound

우리가 알고 싶은 것은 $p(z|x)$를 고려한 $p_\theta(x)$ 를 찾는 것이다. 하나의 데이터 포인트 $x^{(i)}$ 에 대한 log 분포를 잘 변형하면 다음과 같은 식을 유도 할 수 있다.

$$ log p_\theta(x^{(i)}) $$

$$ = \mathbb E_{z\sim q_\phi(z|x^{(i)})} [log p_\theta (x^{(i)})] $$

$$ = \mathbb E_{z\sim q_\phi(z|x^{(i)})} \big[ log \frac{p_\theta (x^{(i)}|z) p_\theta(z)}{p_\theta(z|x^{(i)})} \big]$$

$$ = \mathbb E_{z\sim q_\phi(z|x^{(i)})} \big[ log \frac{p_\theta (x^{(i)}|z) p_\theta(z)}{p_\theta(z|x^{(i)})} \cdot \frac{q_\phi(z|x^{(i)})}{q_\phi(z|x^{(i)})}    \big]$$

$$ = \mathbb E_{z\sim q_\phi(z|x^{(i)})} \big[ log p_\theta(x^{(i)}|z) + log \frac{p_\theta(z)}{q_\phi(z|x^{(i)})} + log \frac{q_\phi(z|x^{(i)})}{p_\theta(z|x^{(i)})} \big]$$

$$ = \mathbb E_{z\sim q_\phi(z|x^{(i)})} \big[ log p_\theta(x^{(i)}|z) \big] + E_{z\sim q_\phi(z|x^{(i)})} \big[log \frac{p_\theta(z)}{q_\phi(z|x^{(i)})} \big] + E_{z\sim q_\phi(z|x^{(i)})} \big[log \frac{q_\phi(z|x^{(i)})}{p_\theta(z|x^{(i)})} \big]$$


- 위 식을 $q_\phi (z|x^{(i)})$ 를 기준으로 Kullback-Leibler Divergence로 정리하면.

$$ = \mathbb E_{z\sim q_\phi(z|x^{(i)})} \big[ log p_\theta(x^{(i)}|z) \big] -D_{KL}\big(q_\phi(z|x^{(i)})|| p_\theta(z) \big) + D_{KL}\big( q_\phi(z|x^{(i)}) || p_\theta(z|x^{(i)}) \big)$$

- 식을 보기 좋게 만들면.

$$\therefore log(p_\theta ( x ^{(i)})) = D_{KL} (q_{\phi}(z|x^{(i)}) || p_\theta (z|x^{(i)})) + \mathcal{L}(\theta, \phi;x^{(i)}) $$


$D_{KL}(\cdot)$ 은 항상 0 이상의 값을 가지기 때문에, Lower bound를 설정할 수 있다.

$$ log(p_\theta ( x ^{(i)})) \ge \mathcal{L}(\theta, \phi;x^{(i)}) = -D_{KL}\big(q_\phi(z|x^{(i)})|| p_\theta(z) \big) + \mathbb E_{z\sim q_\phi(z|x^{(i)})} \big[ log p_\theta(x^{(i)}|z) \big] $$

Lower bound를 최대화하여 $log p_\theta (x^{(i)}) $ 를 최대화 하는 전략을 사용하여 최적화한다.


---
# 3. Reparameterization trick

$z$는 근사된 분포 $q_\phi(z|x)$를 따르는 random하게 sampling된 vector이다.

sampling은 미분이 가능한 연산이 아니므로, $q_\phi(z|x)$를 업데이트하기 위한 backprop을 계산하기 어렵다는 문제가 발생한다. 이를 해결하기 위해, 새로운 함수 $g_\phi (x,\epsilon)$를 이용해 미분가능한 함수로 교묘하게 우회하여 sampling 한다.

![06.png](img/06.png)

**ex)**

예를 들어, $z$가 gaussian 분포를 따른다고 가정 할 때, $q_\phi (z|x)$는 $N(z; \mu, \sigma ^2)$의 sampling일 것이다.

이를 위의 분포에서 직접 sampling 하는 대신, 아래의 식을 통해 값을 sampling 한다. 

- $\mu + \sigma \bigodot \epsilon$

($\epsilon$ 은 $N(0,1)$에서 sampling 된 값,
$\bigodot$ 은 elementwise product)

![08.png](img/08.png)

# 4. Variational Auto-Encoder : Example

Neural network를 이용하여 $\theta, \phi$를 parameter로 사용하는, encoder($q_\phi(z|x))$와 decoder($p_\theta(x|z)$)로 둘 수 있다.

$p(z) = p_\theta(z) = N(z;0,I)$라고 가정할 때, $p_\theta(x|z)$는 multivariate Gaussian으로 생각하고, fully-connected network(decoder)로 계산 할 수 있다.

True posterior인 $p_\theta(z|x)$은 intractable 하지만, gaussain으로 approximate 할 수 있다.

$$ log(q_\theta(z|x^{(i)}) = log N(z; \mu^{(i)}, \sigma^{2(i)}) $$

$mu^{(i)}, \sigma^{2(i)}$는 encoder의 output으로 둘 수 있다.

---
$z^{(i,l)} \sim q_\phi(z|x^{(i)})$일때, Reparameterization trick 을 적용하여, $z^{(i,l)} = g_\phi (x^{(i)}, \epsilon^{(l)}) = \mu^{(i)} + \sigma^{(i)} \cdot \epsilon^{(l)} $
로 둘 수 있다.


In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        # encoder
        self.d1 = nn.Linear(784, 512)
        self.d2 = nn.Linear(512, 256)
        self.d31 = nn.Linear(256, 2)
        self.d32 = nn.Linear(256, 2)
        
        # decoder
        self.d4 = nn.Linear(2, 256)
        self.d5 = nn.Linear(256, 512)
        self.d6 = nn.Linear(512, 784)

    def encode(self, x):
        out = F.relu(self.d1(x))
        out = F.relu(self.d2(out))
        mu = self.d31(out)
        log_var = self.d32(out)
        return mu, log_var
    
    def reparameterization(self, mu, log_var):
        epsilon = torch.randn(mu.shape).to(device)
        std = torch.exp(0.5*log_var)    
        z = std*epsilon + mu
        return z
    
    def decode(self, z):
        out = F.relu(self.d4(z))
        out = F.relu(self.d5(out))
        out = torch.sigmoid(self.d6(out))
        return out
        
    def forward(self, x):
        mu, log_var = self.encode(x.view(-1, 784))
        z = self.reparameterization(mu, log_var)
        
        x_hat = self.decode(z)
        
        return x_hat, mu, log_var


또한 Gaussian case의 경우 Lower bound의 KLD를 직접 계산할 수 있다.
![09](img/09.png)


In [1]:
def loss_func(x, x_hat, mu, log_var):
    bce = F.binary_cross_entropy(x_hat, x.view(-1,784),reduction='sum')
    kld = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return bce + kld